In [1]:
#default_exp training

In [2]:
import os
from deeplearning_image_classification import data_loading

import tensorflow as tf
from tensorflow import keras

gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [3]:
metadata_df = data_loading.get_metadata()

In [4]:
sample_metadata_df = metadata_df.iloc[:1000]

In [5]:
train_dir = os.path.join(data_loading.DATA_DIR, 'images_train/data')

In [6]:
sample_classification_df = data_loading.prepare_classification_metadata(sample_metadata_df, 'Species', train_dir)

In [7]:
image_gen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255, horizontal_flip=True)

sample_image_iterator = image_gen.flow_from_dataframe(sample_classification_df, target_size=(224, 224))

n_classes = len(sample_image_iterator.class_indices)

Found 1000 validated image filenames belonging to 867 classes.


In [8]:
import keras.applications

base_model = keras.applications.MobileNet(include_top=False, input_shape=(224, 224, 3))
model = keras.Sequential(
    [
        base_model,
        keras.layers.AveragePooling2D((4, 4)),
        keras.layers.Flatten(),
        keras.layers.Dense(n_classes),
        keras.layers.Softmax()
    ])
model.compile(loss=keras.losses.CategoricalCrossentropy(), optimizer=keras.optimizers.Adam())

Using TensorFlow backend.


In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Model)   (None, 7, 7, 1024)        3228864   
_________________________________________________________________
average_pooling2d_1 (Average (None, 1, 1, 1024)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 867)               888675    
_________________________________________________________________
softmax_1 (Softmax)          (None, 867)               0         
Total params: 4,117,539
Trainable params: 4,095,651
Non-trainable params: 21,888
_________________________________________________________________


In [10]:
model.fit(sample_image_iterator, epochs=10)

Epoch 1/10
32/32 [==============================] - 22s 699ms/step - loss: 7.0518
Epoch 2/10
32/32 [==============================] - 10s 310ms/step - loss: 6.5989
Epoch 3/10
32/32 [==============================] - 10s 316ms/step - loss: 5.9119
Epoch 4/10
32/32 [==============================] - 10s 312ms/step - loss: 4.4395
Epoch 5/10
32/32 [==============================] - 10s 303ms/step - loss: 2.4360
Epoch 6/10
32/32 [==============================] - 10s 305ms/step - loss: 0.9311
Epoch 7/10
32/32 [==============================] - 10s 307ms/step - loss: 0.4786
Epoch 8/10
32/32 [==============================] - 10s 304ms/step - loss: 0.2747
Epoch 9/10
32/32 [==============================] - 10s 307ms/step - loss: 0.1454
Epoch 10/10
32/32 [==============================] - 10s 306ms/step - loss: 0.1196
